## 作業目標
* 嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [1]:
import os
import keras
# os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

Using TensorFlow backend.
c:\users\user\.conda\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\.conda\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\.conda\envs\tf113\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\.conda\envs\tf113\lib\site-packages\tensorflo

In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin , shape(16,#幾個pixels)
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


In [11]:
print(x_train_histogram.shape) # (50000, 16*3 channels)
x_train_histogram

(50000, 48)


array([[  1.,   4.,   9., ...,   9.,   5.,   0.],
       [ 13.,  22.,  54., ...,  42.,  57.,  35.],
       [  0.,  24.,  87., ...,  15.,  12., 273.],
       ...,
       [ 72., 146., 186., ...,  40.,  67., 168.],
       [ 17.,  11.,  15., ..., 105., 152., 158.],
       [  2.,  20.,  30., ...,  29.,  95.,  37.]], dtype=float32)

#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [4]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

In [12]:
print(x_train_hog.shape) 
x_train_hog

(50000, 64)


array([[9.3837823e+02, 2.0908708e+03, 7.5214545e+02, ..., 3.4196157e+03,
        4.0279946e+03, 2.6420925e+03],
       [9.8362115e+02, 1.1903654e+03, 2.8312205e+03, ..., 5.4915767e+03,
        3.0439009e+03, 5.5831372e+03],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 5.9545195e+03,
        7.6876396e+03, 2.5052759e+03],
       ...,
       [7.4000000e+01, 1.2937971e+02, 1.1229426e+02, ..., 4.2924668e+03,
        3.4137344e+03, 4.0681687e+03],
       [0.0000000e+00, 7.6157737e+00, 0.0000000e+00, ..., 5.7607554e+03,
        2.4398291e+03, 4.0686365e+03],
       [6.7800732e+02, 1.2142119e+03, 2.9075525e+02, ..., 9.5721064e+03,
        5.3560312e+03, 2.4298074e+03]], dtype=float32)

#### SVM model
* SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [該知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

#### 用 histogram 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [5]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

#### 用 HOG 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [6]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

In [7]:
from sklearn.metrics import accuracy_score

print(f'Training data with histogram svm accuracy= {accuracy_score(y_train, y_hist_train)}')
print(f'Testing data with histogram svm accuracy= {accuracy_score(y_test, y_hist_test)}')

print(f'Training data with hog svm accuracy= {accuracy_score(y_train, y_hog_train)}')
print(f'Testing data with hog svm accuracy= {accuracy_score(y_test, y_hog_test)}')


Training data with histogram svm accuracy= 0.14846
Testing data with histogram svm accuracy= 0.1448
Training data with hog svm accuracy= 0.20018
Testing data with hog svm accuracy= 0.1975
